In [2]:
# %load ../start.py
# Load useful extensions

# Activate the autoreload extension for easy reloading of external packages
%reload_ext autoreload
%autoreload 2

# Trun on the water mark
%reload_ext watermark
%watermark -u -d -g

# Load ipycache extension
%reload_ext ipycache
from ipycache import CacheMagics
CacheMagics.cachedir = '../cachedir'

# Add project library to path
import sys
sys.path.insert(0, '../../lib/python')

# The usual suspects
import os
import numpy as np
import pandas as pd

# plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set_context('poster')

# Turn off scientific notation
np.set_printoptions(precision=5, suppress=True)


last updated: 2017-06-14 
Git hash: 5d75a010c3a45348c61a4d9d358243b80eba07e5


In [77]:
from IPython.display import HTML
pd.set_option('display.max_colwidth', -1)

In [4]:
# %load ../../bin/load.py
from pymongo import MongoClient
with open('/home/fearjm/Projects/ncbi_remap/output/.mongodb_host', 'r') as fh:
    host = fh.read().strip()
client = MongoClient(host=host, port=27022)
db = client['sra']
remap = db['remap']
ncbi = db['ncbi']
biometa = db['biometa']

In [6]:
with open('../../data/1508_s2_cell_brian_annot.txt', 'r') as fh:
    s2 = [x.strip() for x in fh]

In [92]:
agg = list(ncbi.aggregate([
    {'$unwind': '$runs'},
    {'$match': {'runs.srr': {'$in': s2}}},
    {
        '$project': {
            '_id': 0,
            'srx': '$srx',
            'srr': '$runs.srr',
            'biosample': '$biosample.biosample_accn',
            'bioproject': '$bioproject.bioproject_accn',
            'pubmed': '$pubmed.pubmed_id',
            'library_strategy': '$sra.experiment.library_strategy',
            'library_source': '$sra.experiment.library_source',
            'instrument_model': '$sra.experiment.instrument_model',
            'study_id': '$runs.study_id',
            'sample_id': '$sra.sample.sample_id',
            'gsm': '$biosample.GEO',
            'title': '$biosample.title',
        }
    },
]))

records = []
for el in agg:
    record = {}
    for k, v in el.items():
        try:
            if isinstance(v, list):
                v = v[0]
        except:
            v = ''
        record[k] = v
    records.append(record)
        

df = pd.DataFrame(records).fillna('')

In [96]:
_mapper = {
    'bioproject': '=HYPERLINK("https://www.ncbi.nlm.nih.gov/bioproject/{0}", "{0}")',
    'biosample': '=HYPERLINK("https://www.ncbi.nlm.nih.gov/biosample/{0}", "{0}")',
    'gsm': '=HYPERLINK("https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc={0}", "{0}")',
    'pubmed': '=HYPERLINK("https://www.ncbi.nlm.nih.gov/pubmed/{0}", "{0}")',
    'srr': '=HYPERLINK("https://trace.ncbi.nlm.nih.gov/Traces/sra/?run={0}", "{0}")',
    'srx': '=HYPERLINK("https://www.ncbi.nlm.nih.gov/sra?term={0}", "{0}")',
}

In [98]:
_df = df.apply(
    lambda x: x.map(
        lambda y: _mapper.get(x.name, '{}').format(y))
).set_index('srx').sort_values('biosample')

In [102]:
_df.to_excel('../../data/1508_s2_cell_jmf_annot.xlsx')

ImportError: No module named 'openpyxl'